# Stack
1. LightGBM
3. XGBoost
4. meta model - Ridge

In [97]:
!pip install -q gdown
!pip install -q category_encoders
!pip install -q optuna
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import gdown
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import OrdinalEncoder, FunctionTransformer, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.linear_model import Ridge
import shap
from sklearn.ensemble import StackingRegressor
import xgboost as xgb
from lightgbm import LGBMRegressor
import category_encoders as ce
from category_encoders import TargetEncoder
import optuna

# Load data

In [98]:
file_id = "1nvFRd8uiUV8OfoihMOXZ0Emle1ksxwW1"
gdown.download(f"https://drive.google.com/uc?id={file_id}", output="uber.csv", quiet=False)
df = pd.read_csv("uber.csv")
df.head()

Downloading...
From: https://drive.google.com/uc?id=1nvFRd8uiUV8OfoihMOXZ0Emle1ksxwW1
To: /content/uber.csv
100%|██████████| 23.5M/23.5M [00:00<00:00, 69.7MB/s]


,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


# Set up
1. Log transform to habdle skew in the target.
2. Drop rows with nans in features without a meaningful imputation value.
3. Establish hold out set.

In [99]:
df.drop_duplicates(inplace=True)

df['log_fare_amount'] = np.log1p(df['fare_amount'])

df = df.dropna(subset=[
    'pickup_latitude',
    'pickup_longitude',
    'dropoff_latitude',
    'dropoff_longitude',
    'pickup_datetime',
    'log_fare_amount'
])

X = df.drop(columns=['fare_amount', 'log_fare_amount', 'key', 'Unnamed: 0'])
y = df['log_fare_amount']

X_train, X_holdout, y_train, y_holdout = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Extraction
### Datetime features extracted (originals dropped)
1. hour
2. weekday
3. month
4. is weekend
5. is rush hour
6. missing datatime flag

If datetime extractor is unable to extract useable datatime, filled with place holder, to prevent crash.

### Coordinate features extracted (originals dropped)
1. haversine distance
2. manhatten distance
3. barring
4. pickup cluster
5. dropoff cluster
6. pickup grid
7. dropoff grid
8. pickup coordinate missing flag
9. dropoff coordinate missing flag

If coordinate extractor is unable to extract useable coordinates, filled with place holder, to prevent crash.

In [100]:
class DatetimeFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, datetime_column='pickup_datetime', add_missing_flag=True):
        self.datetime_column = datetime_column
        self.add_missing_flag = add_missing_flag
        self._output_config = {"transform": "default"}
        self.placeholder = pd.Timestamp("1900-01-01 00:00:00")

    def fit(self, X, y=None):
        self.feature_names_in_ = X.columns.tolist()
        return self

    def transform(self, X):
        check_is_fitted(self)
        X = X.copy()

        X[self.datetime_column] = pd.to_datetime(X[self.datetime_column], errors='coerce')

        if self.add_missing_flag:
            X[f'{self.datetime_column}_missing'] = X[self.datetime_column].isna()

        X[self.datetime_column] = X[self.datetime_column].fillna(self.placeholder)

        X['hour'] = X[self.datetime_column].dt.hour
        X['weekday'] = X[self.datetime_column].dt.weekday
        X['month'] = X[self.datetime_column].dt.month
        X['is_weekend'] = X['weekday'] >= 5
        X['is_rush_hour'] = X['hour'].isin([7, 8, 9, 16, 17, 18, 19])

        X.drop(columns=[self.datetime_column], inplace=True)

        return X

    def set_output(self, *, transform=None):
        self._output_config["transform"] = transform
        return self

class CoordinateFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=20, placeholder=-999.0):
        self.n_clusters = n_clusters
        self.placeholder = placeholder
        self._output_config = {"transform": "default"}
        self.kmeans_pickup = None
        self.kmeans_dropoff = None

    def fit(self, X, y=None):
        pickup_coords = X[['pickup_latitude', 'pickup_longitude']].astype(float)
        dropoff_coords = X[['dropoff_latitude', 'dropoff_longitude']].astype(float)

        self.kmeans_pickup = KMeans(n_clusters=self.n_clusters, random_state=42).fit(pickup_coords)
        self.kmeans_dropoff = KMeans(n_clusters=self.n_clusters, random_state=42).fit(dropoff_coords)

        self.feature_names_in_ = X.columns.tolist()
        return self

    def transform(self, X):
        check_is_fitted(self)
        X = X.copy()

        for col in ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']:
            X[col] = pd.to_numeric(X[col], errors='coerce')

        X['pickup_missing'] = X[['pickup_latitude', 'pickup_longitude']].isna().any(axis=1)
        X['dropoff_missing'] = X[['dropoff_latitude', 'dropoff_longitude']].isna().any(axis=1)

        for col in ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']:
            X[col] = X[col].fillna(self.placeholder)

        X['pickup_latitude'] = X['pickup_latitude'].clip(40.5, 41.0)
        X['pickup_longitude'] = X['pickup_longitude'].clip(-74.3, -73.6)
        X['dropoff_latitude'] = X['dropoff_latitude'].clip(40.5, 41.0)
        X['dropoff_longitude'] = X['dropoff_longitude'].clip(-74.3, -73.6)

        def haversine(lat1, lon1, lat2, lon2):
            R = 6371
            lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
            return R * 2 * np.arcsin(np.sqrt(a))

        def manhattan(lat1, lon1, lat2, lon2):
            return (
                haversine(lat1, lon1, lat2, lon1) +
                haversine(lat2, lon1, lat2, lon2)
            )

        def bearing(lat1, lon1, lat2, lon2):
            dlon = np.radians(lon2 - lon1)
            lat1, lat2 = np.radians(lat1), np.radians(lat2)
            x = np.sin(dlon) * np.cos(lat2)
            y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(dlon)
            return (np.degrees(np.arctan2(x, y)) + 360) % 360

        X['distance_km'] = haversine(
            X['pickup_latitude'], X['pickup_longitude'],
            X['dropoff_latitude'], X['dropoff_longitude']
        )

        X['manhattan_km'] = manhattan(
            X['pickup_latitude'], X['pickup_longitude'],
            X['dropoff_latitude'], X['dropoff_longitude']
        )

        X['bearing'] = bearing(
            X['pickup_latitude'], X['pickup_longitude'],
            X['dropoff_latitude'], X['dropoff_longitude']
        )

        X['pickup_cluster'] = self.kmeans_pickup.predict(X[['pickup_latitude', 'pickup_longitude']])
        X['dropoff_cluster'] = self.kmeans_dropoff.predict(X[['dropoff_latitude', 'dropoff_longitude']])

        X['pickup_grid'] = (
            X['pickup_latitude'].round(3).astype(str) + "_" +
            X['pickup_longitude'].round(3).astype(str)
        )

        X['dropoff_grid'] = (
            X['dropoff_latitude'].round(3).astype(str) + "_" +
            X['dropoff_longitude'].round(3).astype(str)
        )

        X.drop(columns=[
            'pickup_latitude', 'pickup_longitude',
            'dropoff_latitude', 'dropoff_longitude'
        ], inplace=True)

        return X

    def set_output(self, *, transform=None):
        self._output_config["transform"] = transform
        return self

# Feature engineering

In [125]:
class FeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self._output_config = {"transform": "default"}

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        def bin_time(hour):
            if 5 <= hour < 12:
                return 'morning'
            elif 12 <= hour < 17:
                return 'afternoon'
            elif 17 <= hour < 21:
                return 'evening'
            else:
                return 'night'

        X['time_of_day'] = X['hour'].apply(bin_time)
        X['is_peak_hour'] = X['hour'].isin([7, 8, 9, 16, 17, 18, 19])

        X['pickup_dropoff_same_grid'] = (X['pickup_grid'] == X['dropoff_grid']).astype(int)

        X['distance_ratio'] = X['distance_km'] / X['manhattan_km']
        X['distance_ratio'].replace([np.inf, -np.inf], np.nan, inplace=True)
        X['distance_ratio'].fillna(1.0, inplace=True)

        X['cluster_diff'] = np.abs(X['pickup_cluster'] - X['dropoff_cluster'])

        X['distance_bearing_sin'] = X['distance_km'] * np.sin(np.radians(X['bearing']))

        X['cluster_bearing_offset'] = (
            X['dropoff_cluster'] - X['pickup_cluster'] + X['bearing']
        )

        X['hour_x_is_weekend'] = X['hour'] * X['is_weekend']
        X['hour_x_is_rush_hour'] = X['hour'] * X['is_rush_hour']
        X['distance_times_cluster_diff'] = X['distance_km'] * X['cluster_diff']

        return X

    def set_output(self, *, transform=None):
        return self

# Custom classes
1. To float
2. Clip to valid range
3. Cyclic transforer
4. Target encode wrapper

In [113]:
class ToFloat64(BaseEstimator, TransformerMixin):
    def __init__(self):
        self._output_config = {"transform": "default"}

    def fit(self, X, y=None):
        self.feature_names_in_ = X.columns.tolist()
        return self

    def transform(self, X):
        check_is_fitted(self)
        X = X.copy()
        for col in X.columns:
            X[col] = pd.to_numeric(X[col], errors='coerce').astype('float64')
        return X

    def set_output(self, *, transform=None):
        self._output_config["transform"] = transform
        return self

class ClipToValidRange(BaseEstimator, TransformerMixin):
    def __init__(self, valid_min=1, valid_max=6):
        self.valid_min = valid_min
        self.valid_max = valid_max
        self._output_config = {"transform": "default"}

    def fit(self, X, y=None):
        self.feature_names_in_ = X.columns.tolist()
        return self

    def transform(self, X):
        check_is_fitted(self)
        X = X.copy()
        for col in X.columns:
            mask = X[col].between(self.valid_min, self.valid_max)
            X[col] = X[col].where(mask, np.nan)
        return X

    def set_output(self, *, transform=None):
        self._output_config["transform"] = transform
        return self

class CyclicFeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self._output_config = {"transform": "default"}

        self.hour_transformer = FunctionTransformer(lambda X: np.column_stack([
            np.sin(2 * np.pi * X / 24),
            np.cos(2 * np.pi * X / 24)
        ]), validate=False)

        self.bearing_transformer = FunctionTransformer(lambda X: np.column_stack([
            np.sin(np.radians(X)),
            np.cos(np.radians(X))
        ]), validate=False)

        self.weekday_transformer = FunctionTransformer(lambda X: np.column_stack([
            np.sin(2 * np.pi * X / 7),
            np.cos(2 * np.pi * X / 7)
        ]), validate=False)

        self.month_transformer = FunctionTransformer(lambda X: np.column_stack([
            np.sin(2 * np.pi * X / 12),
            np.cos(2 * np.pi * X / 12)
        ]), validate=False)

    def fit(self, X, y=None):
        self.feature_names_in_ = X.columns.tolist()
        return self

    def transform(self, X):
        check_is_fitted(self)
        X = X.copy()
        result = pd.DataFrame(index=X.index)

        if 'hour' in X.columns:
            hour_transformed = self.hour_transformer.transform(X[['hour']])
            result[['hour_sin', 'hour_cos']] = pd.DataFrame(hour_transformed, index=X.index)

        if 'bearing' in X.columns:
            bearing_transformed = self.bearing_transformer.transform(X[['bearing']])
            result[['bearing_sin', 'bearing_cos']] = pd.DataFrame(bearing_transformed, index=X.index)

        if 'weekday' in X.columns:
            weekday_transformed = self.weekday_transformer.transform(X[['weekday']])
            result[['weekday_sin', 'weekday_cos']] = pd.DataFrame(weekday_transformed, index=X.index)

        if 'month' in X.columns:
            month_transformed = self.month_transformer.transform(X[['month']])
            result[['month_sin', 'month_cos']] = pd.DataFrame(month_transformed, index=X.index)

        for col in ['hour', 'bearing', 'weekday', 'month']:
            if col in X.columns:
                X.drop(columns=[col], inplace=True)

        X = pd.concat([X, result], axis=1)

        return X

    def set_output(self, *, transform=None):
        return self

class TargetEncoderWrapper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoder = None
        self.cols = None
        self._output_config = {"transform": "default"}

    def fit(self, X, y=None):
        self.cols = X.columns.tolist()
        self.encoder = ce.TargetEncoder(cols=self.cols)
        self.encoder.fit(X, y)
        return self

    def transform(self, X):
        return self.encoder.transform(X)

    def set_output(self, *, transform=None):
        self._output_config["transform"] = transform
        return self

class FeatureClipper(BaseEstimator, TransformerMixin):
    def __init__(self, clip_range=(0.0, 99.0)):
        self.clip_range = clip_range
        self._output_config = {"transform": "default"}

    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        self.features_ = X.columns.tolist()
        self.lower_bounds_ = {}
        self.upper_bounds_ = {}

        for col in self.features_:
            Q1 = X[col].quantile(self.clip_range[0] / 100)
            Q3 = X[col].quantile(self.clip_range[1] / 100)
            IQR = Q3 - Q1
            self.lower_bounds_[col] = Q1 - 1.5 * IQR
            self.upper_bounds_[col] = Q3 + 1.5 * IQR

        return self

    def transform(self, X):
        check_is_fitted(self, 'features_')
        X = pd.DataFrame(X)

        for col in self.features_:
            X[col] = np.clip(X[col], self.lower_bounds_[col], self.upper_bounds_[col])

        return X

    def set_output(self, *, transform=None):
        return self


# Passenger feature
1. Coerce to float for uniformed nans.
2. Clip range to 1-6 people.
3. Impute with the mode.

In [103]:
passenger_pipeline = Pipeline([
    ('coerce', ToFloat64()),
    ('clip', ClipToValidRange()),
    ('impute', SimpleImputer(strategy='most_frequent')),
])

# Binary features
1. Coerce to float for numeric binarys, 0 1.

In [104]:
binary_pipeline = Pipeline([
    ('coerce', ToFloat64()),
])

# Cyclic features
1. Convert to sinusoidal compenents.

In [105]:
cyclic_pipeline = Pipeline([
    ('cyclic', CyclicFeatureTransformer()),
])

# Ordinal features
1. Encode the clusters.

In [106]:
ordinal_pipeline = Pipeline([
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

# Target encode features
1. Encode grids, safe from leakage.

In [107]:
target_pipeline = Pipeline([
    ('tar', TargetEncoderWrapper())
])

# One hot encode features

In [108]:
ohe_pipeline = Pipeline([
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Distance features
1. Clip extreme values in distance features to avoid skewing features.

In [114]:
distance_pipeline = Pipeline([
    ('clip', FeatureClipper())
])

# Drop low impact features

In [115]:
class DropFeatures(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.features_to_drop = [

]

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        drop_cols = [col for col in self.features_to_drop if col in X.columns]
        return X.drop(columns=drop_cols)

    def set_output(self, *, transform=None):
        return self

# Pipeline

In [127]:
passenger_features = ['passenger_count']
binary_features = ['is_weekend', 'is_rush_hour', 'pickup_missing', 'dropoff_missing', 'pickup_datetime_missing', 'is_peak_hour']
cyclic_features = ['hour', 'bearing', 'weekday', 'month']
target_features = ['pickup_grid', 'dropoff_grid']
ordinal_features = ['pickup_cluster', 'dropoff_cluster']
ohe_features = ['time_of_day']
distance_features = [
    'distance_km',
    'manhattan_km',
    'distance_ratio',
    'cluster_diff',
    'distance_bearing_sin',
    'cluster_bearing_offset',
    'hour_x_is_weekend',
    'hour_x_is_rush_hour',
    'distance_times_cluster_diff',
]

preprocessor = ColumnTransformer([
    ('pass', passenger_pipeline, passenger_features),
    ('cyclic', cyclic_pipeline, cyclic_features),
    ('tar', target_pipeline, target_features),
    ('bin', binary_pipeline, binary_features),
    ('ord', ordinal_pipeline, ordinal_features),
    ('ohe', ohe_pipeline, ohe_features),
    ('dis', distance_pipeline, distance_features),
], remainder='passthrough')

base_learners = [
    ('lgbm', LGBMRegressor(random_state=42, n_jobs=-1)),
    ('xgb', xgb.XGBRegressor(random_state=42, n_jobs=-1, tree_method='hist'))
]

meta_model = Ridge(random_state=42)

stacked_model = StackingRegressor(
    estimators=base_learners,
    final_estimator=meta_model,
    passthrough=False,
    n_jobs=-1
)

full_pipeline = Pipeline([
    ('datetime_features', DatetimeFeatureExtractor(datetime_column='pickup_datetime')),
    ('coordinate_features', CoordinateFeatureExtractor(n_clusters=20)),
    ('feature_engineering', FeatureEngineer()),
    ('preprocessing', preprocessor),
    ('model', stacked_model)
])

full_pipeline.set_output(transform='pandas')

_ = full_pipeline

#Metrics

In [128]:
X_subtrain, X_val, y_subtrain, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

full_pipeline.fit(X_subtrain, y_subtrain)
y_val_pred_log = full_pipeline.predict(X_val)

y_val_true = np.expm1(y_val)
y_val_pred = np.expm1(y_val_pred_log)

val_rmse = np.sqrt(mean_squared_error(y_val_true, y_val_pred))
val_mse = mean_squared_error(y_val_true, y_val_pred)
val_mae = mean_absolute_error(y_val_true, y_val_pred)
val_r2 = r2_score(y_val_true, y_val_pred)

print("Validation Set Metrics (Un-logged, in dollars):")
print(f"RMSE: ${val_rmse:.4f}")
print(f"MSE:  ${val_mse:.4f}")
print(f"MAE:  ${val_mae:.4f}")
print(f"R²:   {val_r2:.4f}")

full_pipeline.fit(X_train, y_train)
y_holdout_pred_log = full_pipeline.predict(X_holdout)

y_holdout_true = np.expm1(y_holdout)
y_holdout_pred = np.expm1(y_holdout_pred_log)

holdout_rmse = np.sqrt(mean_squared_error(y_holdout_true, y_holdout_pred))
holdout_mse = mean_squared_error(y_holdout_true, y_holdout_pred)
holdout_mae = mean_absolute_error(y_holdout_true, y_holdout_pred)
holdout_r2 = r2_score(y_holdout_true, y_holdout_pred)

print("\nHold-Out Set Metrics (Un-logged, in dollars):")
print(f"RMSE: ${holdout_rmse:.4f}")
print(f"MSE:  ${holdout_mse:.4f}")
print(f"MAE:  ${holdout_mae:.4f}")
print(f"R²:   {holdout_r2:.4f}")

Validation Set Metrics (Un-logged, in dollars):
RMSE: $5.2123
MSE:  $27.1681
MAE:  $2.2133
R²:   0.7084

Hold-Out Set Metrics (Un-logged, in dollars):
RMSE: $5.2333
MSE:  $27.3870
MAE:  $2.2063
R²:   0.7119


#Shap feature analysis

In [118]:
X_sampled = X_holdout.sample(n=1000, random_state=42)

X_sampled_transformed = full_pipeline[:-1].transform(X_sampled)

model = full_pipeline.named_steps['model']

explainer = shap.Explainer(model.predict, X_sampled_transformed)
shap_values = explainer(X_sampled_transformed)

mean_abs_shap = np.abs(shap_values.values).mean(axis=0)
shap_importance_df = pd.DataFrame({
    'Feature': X_sampled_transformed.columns,
    'Mean |SHAP Value|': mean_abs_shap
}).sort_values(by='Mean |SHAP Value|', ascending=False).reset_index(drop=True)

print("\nSHAP Feature Importances (All Features):")
print(shap_importance_df)

PermutationExplainer explainer: 1001it [06:35,  2.46it/s]


SHAP Feature Importances (All Features):
                                Feature  Mean |SHAP Value|
0                     clip__distance_km           0.327966
1                     tar__dropoff_grid           0.063032
2                      tar__pickup_grid           0.047075
3          clip__cluster_bearing_offset           0.033566
4                      cyclic__hour_cos           0.032946
5                   cyclic__bearing_sin           0.025823
6            clip__distance_bearing_sin           0.021834
7                  ord__dropoff_cluster           0.019436
8                   cyclic__bearing_cos           0.018480
9                  clip__distance_ratio           0.016964
10                     cyclic__hour_sin           0.015312
11                  ord__pickup_cluster           0.012587
12                   clip__manhattan_km           0.010888
13                  cyclic__weekday_cos           0.010309
14    clip__distance_times_cluster_diff           0.008271
15            

# Summary
The model is generalising well, not overfitting or underfitting, and is accomplishing it's goal well. The top feature is distance followed manhatten distance which makes sense.